In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pickle
import pandas as pd
from pandas import  DataFrame
import numpy as np
import json
import csv
from transformers import BertTokenizer
import tensorflow as tf
import torch
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
import random
import time
import datetime
import plotly.express as px
from sklearn.metrics import matthews_corrcoef, confusion_matrix,accuracy_score,f1_score
from keras.preprocessing.sequence import pad_sequences
from sklearn import metrics
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
from gensim.models import CoherenceModel
#import matplotlib.pyplot as plt
import re
from sklearn.model_selection import GridSearchCV

In [ ]:
#function for getting the frames
def get_frames(df):
    #using cuda as our device
    device = torch.device("cuda")

    #load the model from disk
    model = pickle.load(open('../input/model-for-article/model_article_pickle.sav', 'rb'))
    
    #getting the content 
    test_str_list = df.content.values 
    
    
 
    

    # Tokenize all of the sentences and map the tokens to thier word IDs.
    input_ids = []
    
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

    # For every sentence...
    for st in test_str_list:
        # `encode` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        encoded_sent = tokenizer.encode(
                            st,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                       )

        input_ids.append(encoded_sent)

    # Pad our input tokens
    input_ids = pad_sequences(input_ids, maxlen=128, 
                              dtype="long", truncating="post", padding="post")

    # Create attention masks
    attention_masks = []

    # Create a mask of 1s for each token followed by 0s for padding
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask) 

    # Convert to tensors.
    prediction_inputs = torch.tensor(input_ids)
    prediction_masks = torch.tensor(attention_masks)
   

    # Set the batch size.  
    batch_size = 32  

    # Create the DataLoader.
    prediction_data = TensorDataset(prediction_inputs, prediction_masks)
    prediction_sampler = SequentialSampler(prediction_data)
    prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)
    # Put model in evaluation mode
    model.eval()

    # Tracking variables 
    predictions = []

    # Predict 
    for batch in prediction_dataloader:
            # Add batch to GPU
            batch = tuple(t.to(device) for t in batch)

            # Unpack the inputs from our dataloader
            b_input_ids, b_input_mask= batch

            # Telling the model not to compute or store gradients, saving memory and
            # speeding up validation
            with torch.no_grad():  # Forward pass, calculate logit predictions.
                # This will return the logits rather than the loss because we have
                # not provided labels.
                # token_type_ids is the same as the "segment ids", which
                # differentiates sentence 1 and 2 in 2-sentence tasks.
                # The documentation for this `model` function is here:
                # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
                outputs = model(b_input_ids,
                                token_type_ids=None,
                                attention_mask=b_input_mask)

            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            logits = outputs[0]  # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            
            #print('logits', logits)
           

            #append the predictions (max value index in logits) and true labelsn(label ids)
            predictions.extend((np.argmax(logits, axis=1).flatten()))
    #print('prediction', predictions)
    return predictions
        
   
    
    

In [ ]:
#This function gets the topics
class Topics:
    # second step: we clean the data
    def cleanData(doc):
        texts = []
        # first step of cleaning data is tokenization
        tokenizer = RegexpTokenizer(r'\w+')
        # cleaning one by one for all the documents
        for i in doc:
            # making everything lower case
            i = str(i)
            raw = i.lower()

            # converting into tokens
            tokens = tokenizer.tokenize(raw)
            # print(tokens)
            # removing all the stopwords
            # a list of english stop words
            en_stop = get_stop_words('en')
            # extra from my side
            en_stop.append("s")
            en_stop.append('mr')
            en_stop.append('will')
            en_stop.append('b')
            # compare our tokens with the list of stopwords above
            # remove stop words from tokens
            stopped_tokens = [i for i in tokens if not i in en_stop]
            # print(stopped_tokens)
            # now we perform stemming
            stemmed_tokens = [PorterStemmer().stem(i) for i in stopped_tokens]
            # the tokens are ready to use for the document matrix now...inserting all the stemmed token into list
            texts.append(stemmed_tokens)
            # print('cleaning')
        print('cleaned')
        return texts


    # The third part is constructing a document term matrix.....(text to word)
    def constructDocMatrix(texts):
        # we need to check how frequent a word appears in each document
        # Dicinoary() iterates through each word in text, giving unique id to them and collects data such as count
        dictionary = corpora.Dictionary(texts)
        # now our dictionary must be converted into bag of words
        # doc2bow converts dictinoary into bag of words
        corpus = [dictionary.doc2bow(text) for text in texts]
        print('matrix complete')
        return corpus, dictionary


    # model evaluation; now we calculate the coherence to find the optimum number of k in the doc
    def checkCoherence(corpus, texts, dictionary, min_k, max_k, interv):
        coherence = []
        k_list = []
        for k in range(min_k, max_k, interv):
            print('checking')
            ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=k, random_state=2, id2word=dictionary, passes=15)
            coherence_model_lda = CoherenceModel(model=ldamodel, texts=texts, dictionary=dictionary, coherence='c_v')
            coherence_lda = coherence_model_lda.get_coherence()
            coherence.append(coherence_lda)
            k_list.append(k)
        return coherence, k_list





    # find the dominant topic in each document
    def findDomaninant(ldamodel, corpus, i):
        topic_weight = []
        dominant_topic_ = ''
        # sort the topic weight in descending order and choose the first one i.e highest weight
        topics = ldamodel[corpus[i]]
        for topic in topics:
            topic_weight.append(topic[1])
            topic_weight.sort(reverse=True)
        for topic in topics:
            if topic[1] == topic_weight[0]:
                dominant_topic_ = topic[0]
        # print(dominant_topic)
        topic_weight.clear()
        return dominant_topic_

    

    

In [ ]:
def main():
    #importing the webis corpus json file 
    df = pd.read_json('../input/webis-json-file/articles_with_adu_liwc_lexicons_content.json', orient='records')
    #df.set_index('idx', inplace=True)
    #get frames for 979 editorials
    frames_list = get_frames(df)
    df['frame'] = frames_list
    #print(df['frames'])
    #get topics for 979 editorials
    texts = Topics.cleanData(df['content'])
    corpus, dictionary = Topics.constructDocMatrix(texts)
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=40, random_state=2, id2word=dictionary, passes=15)
    topics = ldamodel.print_topics(num_topics=40, num_words=3)
    list_topics_graph = []
    for topic in topics:
        # print(topic[1])
        list_topics_graph.append(re.sub('[^a-zA-Z]+', '-', topic[1]))
    print(list_topics_graph)
    # findDomaninant(ldamodel, corpus, 0)
    dominant_topic_list = []
    dominant_topic = ''
    #getting the dominant topics for each editorials from the corpus
    for i in range(len(ldamodel[corpus])):
        dominant_topic = Topics.findDomaninant(ldamodel, corpus, i)
        dominant_topic_list.append(dominant_topic)
    
    df['topic'] = dominant_topic_list
    print(df['topic'])
    df.to_json('new_articles_with_adu_liwc_lexicons_content.json')
         
    

if __name__=='__main__':
    main()

In [ ]:
#Import svm model
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler
from sklearn.svm import SVC  


def svc_param_gridsearch(X, y, nfolds_or_division):
    Cs = [  0.01, 0.1, 1, 10, 100]
    param_grid = {'C': Cs, 'class_weight': ['balanced']}#, 'gamma' : gammas}
    grid_search = GridSearchCV(SVC(kernel='linear'), param_grid, cv=nfolds_or_division, n_jobs=2, scoring='f1_macro')
    
    grid_search.fit(X, y)
    
    return grid_search.best_params_#, grid_search.best_score_

def train_save(X_train, y_train, params={}):
    # 'linear', 'poly', 'rbf', 'sigmoid', 'precomputed'
    print('params ', str(params))
    cw = params['class_weight'] if 'class_weight' in params else None
    c = params['C'] if 'C' in params else 'balanced'
    clf = svm.SVC(kernel='linear', class_weight=cw, C=c)  
    return clf

def normalize(X_train, X_test, normalizing_method="standard"):
    if normalizing_method == "standard":
        print("Normalizing by using standard scaler...")
        scaler = StandardScaler(copy=True, with_mean=False)
        scaler.fit(X_train)
        X_train = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)
        X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns) if (X_test is not None) else None
    elif normalizing_method == "log":    
        X_train = np.log(X_train+1)
        X_test = np.log(X_test+1) if (X_test is not None) else None
    elif normalizing_method == "sqrt":
        X_train = np.sqrt(X_train+(2/3))
        X_test = np.sqrt(X_test+(2/3)) if (X_test is not None) else None
    elif normalizing_method == "minmax":
        scaler = MinMaxScaler(copy=True)
        scaler.fit(X_train)

        X_train = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)
        X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns) if (X_test is not None) else None
    else:
        print("data is not normalized. method is not supported. Choose one of: standard, minmax, log, sqrt")

    return X_train, X_test

#importing the new updated dataframe with topics and frames
df1 = pd.read_json('../input/new-webis-json-file/new_articles_with_adu_liwc_lexicons_content(1).json', orient='records')

#print(df1['mpqa_subjobg_subj'])
#print(df1["mpqa_subjobg_subj"].describe()) 
df1['mpqa_subjobg_subj'] = df1['mpqa_subjobg_subj'].fillna(0)
#print(df1.isnull().sum())
#lists of combination for the best model (liberal)
liwc, mpqa_sub, mpqa_arg, content = [],[],[],[]
#style features
liwc = [l for l in df1.columns if l.startswith('liwc_')]
mpqa_arg = [l for l in df1.columns if l.startswith('mpqa_arg_')]
mpqa_sub = [l for l in df1.columns if l.startswith('mpqa_subjobg_')]
nrc_emo_sent =  [l for l in df1.columns if l.startswith('nrc_')]
webis_adu =  [l for l in df1.columns if l.startswith('adu_')]
#content
content = [l for l in df1.columns if l.startswith('lemma_')]
#split label
split = [l for l in df1.columns if l.startswith('split')]
#majority label
liberal = [l for l in df1.columns if l.startswith('liberal')]
conservative = [l for l in df1.columns if l.startswith('conservative')]
topics = df1.topic.values
frames = df1.frame.values
#print(df1.topic.value_counts())
#print(df1.frame.value_counts())

df2 = pd.DataFrame()
#adding all the required columns (combinations) to new dataframe
#for a in liwc:
    #df2[a] = df1[a]
for b in mpqa_arg:
    df2[b] = df1[b]
#for c in mpqa_sub:
    #df2[c] = df1[c]
for d in content:
    df2[d] = df1[d]
for e in split:
    df2[e] = df1[e]
#for i in nrc_emo_sent:
    #df2[i] = df1[i]
#for j in webis_adu:
    #df2[j] = df1[j]
#for f in liberal:
    #df2[f] = df1[f]
for g in conservative:
    df2[g] = df1[g]
#df2['topic'] = topics
df2['frame'] = frames

#now we create 2 dataframes (for train and test) and add all these features to it
df_train = df2.loc[df2['split_label'] == 'train']
df_test = df2.loc[df2['split_label'] == 'test']
#print(df_train.shape)
#print(df_test.shape)


#this will concat 2 dataframes without losing their index
df_merge = pd.concat([df_train, df_test])
#print(df_merge['split_label'][-200:])

target = df_merge.conservative_majority.values
df_merge = df_merge.drop([ 'split_label','conservative_majority'], axis=1)



#print(df_merge.isnull().sum())

#now we use sk learn train test split, without suffling and split (783-196) just as in the paper
X_train, X_test, y_train, y_test = train_test_split(df_merge,target, test_size=196,shuffle = False)

#X_train, X_test = normalize(X_train, X_test, normalizing_method='sqrt')
best_params = svc_param_gridsearch(X_train, y_train, nfolds_or_division=5)

clf_trained = train_save(X_train, y_train, params=best_params)

#Train the model using the training sets
clf_trained.fit(X_train, y_train)


#Predict the response for test dataset
y_pred = clf_trained.predict(X_test)

# Model Accuracy: how often is the classifier correct?
#print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

print(f1_score(y_test, y_pred, average='micro'))
print(f1_score(y_test, y_pred, average='macro'))

'''fix test_size and nan value in the last column'''